# Wrangle

In [89]:
#imports
import pandas as pd
import numpy as np

# system
import sys
# sys.path.append("../data")
sys.path.append("util_")
import wrangle_

In [90]:
# lacd data
df = pd.read_csv("data/known_exploited_vulnerabilities.csv")
df.head()

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,notes,cwes
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,NaN,NaN


In [3]:
# look at the shape
df.shape

(1140, 11)

In [4]:
#look at the columns
df.columns

Index(['cveID', 'vendorProject', 'product', 'vulnerabilityName', 'dateAdded',
       'shortDescription', 'requiredAction', 'dueDate',
       'knownRansomwareCampaignUse', 'notes', 'cwes'],
      dtype='object')

In [5]:
# check the full info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cveID                       1140 non-null   object 
 1   vendorProject               1140 non-null   object 
 2   product                     1140 non-null   object 
 3   vulnerabilityName           1140 non-null   object 
 4   dateAdded                   1140 non-null   object 
 5   shortDescription            1140 non-null   object 
 6   requiredAction              1140 non-null   object 
 7   dueDate                     1140 non-null   object 
 8   knownRansomwareCampaignUse  1140 non-null   object 
 9   notes                       373 non-null    object 
 10  cwes                        0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 98.1+ KB


In [6]:
# select numeric
len(df.select_dtypes("number").columns) # count of numeric columns

1

In [7]:
#select object columns
len(df.select_dtypes("object").columns) #count of object columns

10

In [8]:
df.describe()

,cwes
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


**What I see:**


- I have 1140 rows and 11columns
- 1 of the 11 columns are numric while 10 of them are string object colums
- the numeric column `cwes` is null
- the `notes` column has 373 non null rows
- descriptive statistics says nothing on the `cwes` empty column

In [10]:
# remove the non useul unique id and the emplt column
df = df.drop(columns=["cveID", "dateAdded","cwes"])
df.columns

Index(['vendorProject', 'product', 'vulnerabilityName', 'shortDescription',
       'requiredAction', 'dueDate', 'knownRansomwareCampaignUse', 'notes'],
      dtype='object')

In [11]:
# nanipulate the column names to a clear name
spaced_cols = []
col_renamed = ""
for name in df.columns:
    for ele in name:
        if ele == ele.upper():
            col_renamed += " " + ele
        else:
            col_renamed += ele
    spaced_cols.append(col_renamed)
    col_renamed = ""

df.columns = spaced_cols
spaced_cols

['vendor Project',
 'product',
 'vulnerability Name',
 'short Description',
 'required Action',
 'due Date',
 'known Ransomware Campaign Use',
 'notes']

In [12]:
# remove spaces from the ends
# replace " " with _
# convert from lower case
new_cols = df.columns.str.strip().str.replace(" ", "_").str.lower()
df.columns = new_cols #replace the original columns with the new columns
df.columns

Index(['vendor_project', 'product', 'vulnerability_name', 'short_description',
       'required_action', 'due_date', 'known_ransomware_campaign_use',
       'notes'],
      dtype='object')

In [13]:
# remove duplicated rows
print("original data size: ", df.shape)
df = df.drop_duplicates(keep="first")
print("new data size: ", df.shape)

original data size:  (1140, 8)
new data size:  (1099, 8)


In [16]:
# Split the data into 3 sets
train, validate, test = wrangle_.split_data_(df=df, 
                     test_size=0.2, 
                     validate_size= 0.2, 
                     stratify_col="known_ransomware_campaign_use", 
                     random_state=10)
train.shape, validate.shape, test.shape

((659, 8), (220, 8), (220, 8))

In [17]:
# reset index
train = train.reset_index(drop=True)

In [18]:
# count values for each feature
for col in train.columns:
    print(train[col].value_counts)
    print("unique value count size:",train[col].value_counts().shape)
    print("\n\n")

<bound method IndexOpsMixin.value_counts of 0          Zimbra
1          Google
2      SolarWinds
3         Mozilla
4           Rails
          ...    
654         Adobe
655     Hikvision
656     Microsoft
657         Cisco
658        Google
Name: vendor_project, Length: 659, dtype: object>
unique value count size: (121,)



<bound method IndexOpsMixin.value_counts of 0                         Collaboration (ZCS)
1                                 Chromium V8
2                                      Serv-U
3                     Firefox and Thunderbird
4                               Ruby on Rails
                        ...                  
654                                ColdFusion
655               Security cameras web server
656                         Internet Explorer
657    IOS Software and Cisco IOS XE Software
658                               Chromium V8
Name: product, Length: 659, dtype: object>
unique value count size: (308,)



<bound method IndexOpsMixin.value_counts of 0

# Tasks from EDA

* The source of memory corruption is from 4 vendors, but it looks like it's happening on different products of these vendors, so I will go back inthe wrangle file and separate the product from the volunerability and create a additional column for specific vulnerabilities groupd by wording. (cell 7 - cell 15)

In [37]:
train = pd.read_csv("./data/00_temp_train_data.csv", index_col=0)
# create a copy of the training data
temp_train = train.copy()

In [57]:
# Engineer a new feature `vulnerability types` that is the grouping of all alike valnerability names into single names. 
vulnerability_types = {
    'Arbitrary File Upload Vulnerability': [
        'Zimbra Collaboration (ZCS) Arbitrary File Upload Vulnerability',
        'SonicWall Email Security Unrestricted Upload of File Vulnerability',
        'Adobe ColdFusion Unrestricted File Upload Vulnerability',
    ],
    'Out-of-Bounds Write Vulnerability': [
        'Google Chromium V8 Out-of-Bounds Write Vulnerability',
        'Apple iOS and macOS Out-of-Bounds Write Vulnerability',
        'VMware vCenter Server Out-of-Bounds Write Vulnerability',
        'Adobe Acrobat and Reader Out-of-Bounds Write Vulnerability',
        'Exim Out-of-bounds Write Vulnerability',
        'SAP NetWeaver Out-of-Bounds Write Vulnerability'
        'Microsoft Windows VBScript Engine Out-of-Bounds Write Vulnerability',
        'Apple iOS, iPadOS, and watchOS Out-of-Bounds Write Vulnerability',
        'Android Kernel Out-of-Bounds Write Vulnerability',
        'Fortinet FortiOS Out-of-Bound Write Vulnerability'
    ],
    'Path Traversal Vulnerability': [
        'SolarWinds Serv-U Path Traversal Vulnerability',
        'IBM Data Risk Manager Directory Traversal Vulnerability',
        'SAP Customer Relationship Management (CRM) Path Traversal Vulnerability',
        'Citrix NetScaler ADC and NetScaler Gateway Path Traversal Vulnerability',
        'TVT NVMS-1000 Directory Traversal Vulnerability',
        'QNAP Photo Station Path Traversal Vulnerability',
        'Juniper Junos OS Path Traversal Vulnerability',
        'Cisco IOS Software Integrated Services Module for VPN Path Traversal Vulnerability',
        'Apache HTTP Server Path Traversal Vulnerability'
        'SolarWinds Serv-U Path Traversal Vulnerability',
        'Ruby on Rails Directory Traversal Vulnerability',
        'Cisco ASA and FTD Read-Only Path Traversal Vulnerability',
        'Cisco Prime Data Center Network Manager (DCNM) Directory Traversal Vulnerability',
        'Adobe ColdFusion Directory Traversal Vulnerability',
        'SaltStack Salt Path Traversal Vulnerability',
        'Microsoft Windows TS WebProxy Directory Traversal Vulnerability'
    ],
    'Use-After-Free Vulnerability': [
        'Mozilla Firefox and Thunderbird Use-After-Free Vulnerability',
        'Google Chromium WebGL Use-After-Free Vulnerability',
        'Adobe Flash Player Use-After-Free Vulnerability',
        'Google Chromium V8 Use-After-Free Vulnerability',
        'Apple iOS, iPadOS, and watchOS WebKit Use-After-Free Vulnerability',
        'Google Chrome WebAudio Use-After-Free Vulnerability',
        'Microsoft Internet Explorer Use-After-Free Vulnerability',
        'Apple Multiple Products Use-After-Free Vulnerability'
    ],
    'Remote Code Execution Vulnerability': [
        'VMware vCenter Server Remote Code Execution Vulnerability',
        'Microsoft Word Remote Code Execution Vulnerability',
        'Microsoft MSHTML Remote Code Execution Vulnerability',
        'Oracle WebLogic Server Remote Code Execution Vulnerability',
        'Zoho ManageEngine ServiceDesk Plus Remote Code Execution Vulnerability',
        'SAP NetWeaver Remote Code Execution Vulnerability',
        'Apache Tomcat Remote Code Execution Vulnerability',
        'Unraid Remote Code Execution Vulnerability',
        'Microsoft Exchange Server Remote Code Execution Vulnerability',
        'Microsoft Office Remote Code Execution Vulnerability',
        'Google Chromium Network Service Use-After-Free Vulnerability',
        'Microsoft Remote Desktop Services Remote Code Execution Vulnerability',
        'Sitecore XP Remote Command Execution Vulnerability',
        'Veritas Backup Exec Agent Command Execution Vulnerability',
        'Fortra GoAnywhere MFT Remote Code Execution Vulnerability',
        'XStream Remote Code Execution Vulnerability',
        'Adobe Flash Player Remote Code Execution Vulnerability',
        'Microsoft Office Remote Code Execution Vulnerability'
    ],
    'Improper Access Control Vulnerability': [
        'Citrix Content Collaboration ShareFile Improper Access Control Vulnerability',
        'Fortinet FortiOS and FortiADC Improper Access Control Vulnerability',
        'QNAP Helpdesk Improper Access Control Vulnerability',
        'Trend Micro Multiple Products Improper Access Control Vulnerability',
        'PaperCut MF/NG Improper Access Control Vulnerability',
        'Citrix ShareFile Improper Access Control Vulnerability',
        'MiCollab, MiVoice Business Express Access Control Vulnerability',
        'Samsung Mobile Devices Improper Access Control Vulnerability',
        'SonicWall Email Security Improper Privilege Management Vulnerability',
        'Microsoft Windows Mark of the Web (MOTW) Security Feature Bypass Vulnerability',
        'VMware vCenter Server and Cloud Foundation Server Side Request Forgery (SSRF) Vulnerability',
        'Trend Micro Multiple Products Improper Input Validation Vulnerability',
        'Telerik UI for ASP.NET AJAX Insecure Direct Object Reference Vulnerability',
        'Microsoft Malware Protection Engine Improper Restriction of Operations Vulnerability',
        'GitLab Community and Enterprise Editions Improper Access Control Vulnerability',
        'Adobe ColdFusion Improper Access Control Vulnerability'
    ],
    'Privilege Escalation Vulnerability': [
        'Microsoft Windows Kernel Privilege Escalation Vulnerability',
        'Microsoft Windows Privilege Escalation Vulnerability',
        'Microsoft Windows CSRSS Security Feature Bypass Vulnerability',
        'Microsoft Win32k Privilege Escalation Vulnerability',
        'Microsoft Windows Hyper-V Privilege Escalation Vulnerability',
        'Microsoft Windows Advanced Local Procedure Call (ALPC) Privilege Escalation Vulnerability',
        'Microsoft Windows AppX Deployment Extensions Privilege Escalation Vulnerability',
        'GIGABYTE Multiple Products Privilege Escalation Vulnerability',
        'Apple iOS, iPadOS, and macOS Kernel Privilege Escalation Vulnerability',
        'Microsoft Windows Common Log File System (CLFS) Driver Privilege Escalation Vulnerability',
        'Microsoft Windows Installer Privilege Escalation Vulnerability',
        'Microsoft Windows Print Spooler Privilege Escalation Vulnerability',
        'Microsoft Windows AppX Deployment Server Privilege Escalation Vulnerability',
        'Microsoft Windows Enhanced Cryptographic Provider Privilege Escalation Vulnerability',
        'Microsoft Windows User Profile Service Privilege Escalation Vulnerability',
        'Microsoft Windows MSHTML Platform Privilege Escalation Vulnerability',
        'Microsoft Windows Common Log File System (CLFS) Driver Privilege Escalation Vulnerability',
        'Microsoft Windows Task Scheduler Privilege Escalation Vulnerability',
        'Microsoft Exchange Server Privilege Escalation Vulnerability',
        'Microsoft Skype for Business Privilege Escalation Vulnerability',
        'Microsoft Windows Cloud Files Mini Filter Driver Privilege Escalation Vulnerability',
        'Microsoft .NET Framework Privilege Escalation Vulnerability',
        'Microsoft Windows AppX Deployment Extensions Privilege Escalation Vulnerability',
        'SolarWinds Virtualization Manager Privilege Escalation Vulnerability',
        'Microsoft Windows Secondary Logon Service Privilege Escalation Vulnerability',
        'Microsoft Windows Privilege Common Log File System (CLFS) Escalation Vulnerability',
        'Microsoft Windows Group Policy Preferences Password Privilege Escalation Vulnerability',
        'Microsoft Windows Mount Manager Privilege Escalation Vulnerability',
        'Microsoft Streaming Service Proxy Privilege Escalation Vulnerability',
        'Microsoft Win32k Privilege Escalation Vulnerability'
    ],
    'Buffer Overflow Vulnerability': [
        'Microsoft PowerPoint Buffer Overflow Vulnerability',
        'Adobe Acrobat and Reader Stack-Based Buffer Overflow Vulnerability',
        'Adobe Reader Buffer Overflow Vulnerability',
        'Adobe Acrobat and Reader Buffer Overflow Vulnerability',
        'PHP FastCGI Process Manager (FPM) Buffer Overflow Vulnerability',
        'Cisco Adaptive Security Appliance (ASA) SNMP Buffer Overflow Vulnerability',
        'D-Link DIR-825 R1 Devices Buffer Overflow Vulnerability',
        'Cisco Small Business RV Series Routers Stack-based Buffer Overflow Vulnerability',
        'F5 BIG-IP Traffic Management Microkernel Buffer Overflow',
        'Adobe Flash Player Stack-based Buffer Overflow Vulnerability',
        'MikroTik RouterOS Stack-Based Buffer Overflow Vulnerability',
        'NETGEAR Multiple Devices Buffer Overflow Vulnerability'
    ],
    'SQL Injection Vulnerability': [
        'Sophos SFOS SQL Injection Vulnerability',
        'Citrix NetScaler ADC and NetScaler Gateway SQL Injection Vulnerability',
        'SAP NetWeaver SQL Injection Vulnerability',
        'QNAP VioStor NVR OS Command Injection Vulnerability',
        'Primetek Primefaces Remote Code Execution Vulnerability',
        'SonicWall SSLVPN SMA100 SQL Injection Vulnerability',
        'Grandstream Networks UCM6200 Series SQL Injection Vulnerability',
        'Citrix SD-WAN and NetScaler SQL Injection Vulnerability',
        'F5 BIG-IP Configuration Utility SQL Injection Vulnerability'
    ],
    'Command Injection Vulnerability': [
        'Webmin Command Injection Vulnerability',
        'Apache Kylin OS Command Injection Vulnerability',
        'Cisco IOS and IOS XE Software Command Injection Vulnerability',
        'SolarView Compact Command Injection Vulnerability',
        'NETGEAR Multiple WAP Devices Command Injection Vulnerability',
        'D-Link DIR-859 Router Command Execution Vulnerability',
        'TP-Link Multiple Archer Devices Directory Traversal Vulnerability',
        'Zyxel EMG2926 Routers Command Injection Vulnerability',
        'Cisco NX-OS Command Injection Vulnerability',
        'Ivanti Pulse Connect Secure and Policy Secure Command Injection Vulnerability',
        'Crestron Multiple Products Command Injection Vulnerability',
        'Vmware Aria Operations for Networks Command Injection Vulnerability',
        'Zyxel Multiple NAS Devices Command Injection Vulnerability',
        'D-Link DSL-2750B Devices Command Injection Vulnerability',
        'Apache RocketMQ Command Execution Vulnerability',
        'rConfig OS Command Injection Vulnerability',
        'SaltStack Salt Shell Injection Vulnerability'
    ],
    'Denial of Service Vulnerability': [
        'Cisco IOS Software Network Address Translation Denial-of-Service Vulnerability',
        'Cisco ASA and FTD Denial of Service Vulnerability',
        'Apache Struts 1 ActionForm Denial-of-Service Vulnerability',
        'Cisco IOS XR Software DVMRP Memory Exhaustion Vulnerability',
        'Cisco IOS and IOS XE Software DHCP Remote Code Execution Vulnerability',
        'Intel Ethernet Diagnostics Driver for Windows Denial-of-Service Vulnerability',
        'Google Chromium Intents Improper Input Validation Vulnerability',
        'Cisco IOS and IOS XE Software Internet Key Exchange Denial-of-Service Vulnerability',
        'Service Location Protocol (SLP) Denial-of-Service Vulnerability',
        'Cisco IOS Software Integrated Services Module for VPN Denial-of-Service Vulnerability',
        'Cisco IOS Software and Cisco IOS XE Software UDP Packet Processing Denial-of-Service Vulnerability'
    ],
    'Information Disclosure Vulnerability': [
        'Mozilla Firefox Information Disclosure Vulnerability',
        'Microsoft Internet Explorer Information Disclosure Vulnerability',
        'Oracle WebLogic Server Information Disclosure Vulnerability',
        'SAP NetWeaver Information Disclosure Vulnerability',
        'Microsoft Windows Graphics Device Interface (GDI) Information Disclosure Vulnerability',
        'Cisco IOS, IOS XR, and IOS XE IKEv1 Information Disclosure Vulnerability',
        'Red Hat JBoss Information Disclosure Vulnerability',
        'Cisco Small Business RV320 and RV325 Routers Information Disclosure Vulnerability',
        'Google Chromium Information Disclosure Vulnerability',
        'Microsoft Exchange Server Information Disclosure',
        'Cisco ASA and FTD Information Disclosure Vulnerability'
    ],
    'Deserialization of Untrusted Data Vulnerability': [
        'Apache Solr VelocityResponseWriter Plug-In Remote Code Execution Vulnerability',
        'Apache Log4j2 Deserialization of Untrusted Data Vulnerability',
        'Jenkins Stapler Web Framework Deserialization of Untrusted Data Vulnerability',
        'NextGen Healthcare Mirth Connect Deserialization of Untrusted Data Vulnerability',
        'Apache Struts Deserialization of Untrusted Data Vulnerability',
        'Adobe ColdFusion Deserialization of Untrusted Data Vulnerability',
        'Progress WS_FTP Server Deserialization of Untrusted Data Vulnerability',
        'Progress Telerik UI for ASP.NET AJAX Deserialization of Untrusted Data Vulnerability',
        'Cisco RV Series Routers Deserialization of Untrusted Data Vulnerability',
        'Apache ActiveMQ Deserialization of Untrusted Data Vulnerability',
        'Kentico Xperience Deserialization of Untrusted Data Vulnerability',
    ],
    'Sandbox Bypass Vulnerability': [
        'Jenkins Script Security Plugin Sandbox Bypass Vulnerability',
        'Adobe Acrobat and Reader Sandbox Bypass Vulnerability',
        'Adobe Flash Player Sandbox Bypass Vulnerability',
        'Oracle Java SE Sandbox Bypass Vulnerability',
        'Oracle JRE Sandbox Bypass Vulnerability',
        'Debian-specific Redis Server Lua Sandbox Escape Vulnerability'
    ],
    'Arbitrary Code Execution Vulnerability': [
        'Adobe Flash Player Arbitrary Code Execution Vulnerability',
        'Adobe ColdFusion Arbitrary Code Execution Vulnerability',
        'Microsoft Office Arbitrary Code Execution Vulnerability',
        'Oracle Java SE Runtime Environment (JRE) Arbitrary Code Execution Vulnerability',
        'Cisco IOS and IOS XE Software SNMP Remote Code Execution Vulnerability',
        'Microsoft Hyper-V RemoteFX vGPU Remote Code Execution Vulnerability',
        'Microsoft Windows Remote Code Execution Vulnerability',
        'Microsoft .NET Framework Remote Code Execution Vulnerability',
        'Palo Alto Networks PAN-OS Remote Code Execution Vulnerability',
        'Microsoft Internet Explorer Remote Code Execution Vulnerability',
        'Jenkins Matrix Project Plugin Remote Code Execution Vulnerability',
        'GitLab Community and Enterprise Editions Remote Code Execution Vulnerability',
        'Fortra Cobalt Strike User Interface Remote Code Execution Vulnerability',
        'Sonatype Nexus Repository Remote Code Execution Vulnerability',
        'Kaseya VSA Remote Code Execution Vulnerability',
        'VMware Workspace ONE Access and Identity Manager Server-Side Template Injection Vulnerability',
        'Rejetto HTTP File Server (HFS) Remote Code Execution Vulnerability',
        'D-Link DNS-320 Remote Code Execution Vulnerability',
        'Apple Multiple Products Code Execution Vulnerability',
        'ThinkPHP "noneCms" Remote Code Execution Vulnerability'
    ],
    'Code Injection Vulnerability': [
        'Apache Solr DataImportHandler Code Injection Vulnerability',
        'Citrix NetScaler ADC and NetScaler Gateway Code Injection Vulnerability',
        'VMware Spring Cloud Gateway Code Injection Vulnerability',
        'Mitel MiVoice Connect Code Injection Vulnerability',
        'Crestron Multiple Products Command Injection Vulnerability',
        'PHPUnit Command Injection Vulnerability',
        'Vmware Aria Operations for Networks Command Injection Vulnerability',
        'Apple iOS Wallet Code Execution Vulnerability'
    ],
    'Type Confusion Vulnerability': [
        'Google Chromium V8 Type Confusion Vulnerability',
        'Google Chrome FreeType',
        'Type Confusion Vulnerability',
        'Apple Multiple Products Type Confusion Vulnerability',
        'Adobe Flash Player Type Confusion Vulnerability',
        'Microsoft Internet Explorer Type Confusion Vulnerability',
        'Apple iOS Type Confusion Vulnerability',
        'Google Chromium V8 Incorrect Implementation Vulnerability',
    ],
    'Memory Corruption Vulnerability': [
        'Apple iOS, iPadOS, and macOS Memory Corruption Vulnerability',
        'Microsoft Edge and Internet Explorer Memory Corruption Vulnerability',
        'Google Chromium V8 Memory Corruption Vulnerability',
        'Adobe Flash Player Memory Corruption Vulnerability',
        'Mozilla Firefox Memory Corruption Vulnerability',
        'Microsoft Windows Memory Corruption Vulnerability',
        'Apple Memory Corruption Vulnerability',
        'Microsoft Internet Explorer Memory Corruption Vulnerability',
        'Microsoft Office Memory Corruption Vulnerability',
        'Apple Multiple Products Memory Corruption Vulnerability',
        'Microsoft Win32k Memory Corruption Vulnerability',
        'Adobe Reader and Acrobat Memory Corruption Vulnerability',
        'Microsoft Graphics Component Memory Corruption Vulnerability',
        'Apple iOS Webkit Memory Corruption Vulnerability'
    ],
    'Heap Buffer Overflow Vulnerability': [
        'Google Chromium V8 Heap Buffer Overflow Vulnerability',
        'Microsoft Windows Kernel Heap Buffer Overflow Vulnerability',
        'Apple iOS and macOS Heap Buffer Overflow Vulnerability',
        'Adobe Flash Player Heap Buffer Overflow Vulnerability',
        'WebRTC Heap Buffer Overflow Vulnerability'
    ],
    'Arbitrary File Download Vulnerability': [
        'Apple iOS, iPadOS, and macOS Arbitrary File Download Vulnerability'
    ],
    'Stack-Based Buffer Overflow Vulnerability': [
        'Cisco IOS and IOS XE Software Stack-Based Buffer Overflow Vulnerability',
        'D-Link DIR-825 R1 Devices Stack-Based Buffer Overflow Vulnerability',
        'Mitel MiVoice Connect Stack-Based Buffer Overflow Vulnerability'
    ],
    'Improper Restriction of Operations Vulnerability': [
        'SAP NetWeaver Improper Restriction of Operations Vulnerability',
        'Apple iOS and iPadOS Improper Restriction of Operations Vulnerability',
        'Microsoft Malware Protection Engine Improper Restriction of Operations Vulnerability'
    ],
    'Insecure Default Initialization of Resource Vulnerability': [
        'IBM QRadar SIEM Insecure Default Initialization of Resource Vulnerability'
    ],
    'Unspecified Vulnerability': [
        'Oracle WebLogic Server Unspecified Vulnerability',
        'Adobe Flash Player Unspecified Vulnerability',
        'Apple Multiple Products Kernel Unspecified Vulnerability',
        'Samsung Mobile Devices Unspecified Vulnerability',
        'Apple iOS, iPadOS, and watchOS Unspecified Vulnerability'
    ]
}

In [58]:
# Replacing values in the "vulnerability names" column based on the specific vulnerabilties
def replace_vulnerability_name(name):
    for key, value in vulnerability_types.items():
        if name in value:
            return key
    return 'Other'

# apply the created function to the data
temp_train["vulnerability_type"] = temp_train["vulnerability_name"].apply(replace_vulnerability_name)

In [59]:
temp_train.vulnerability_type.value_counts()

vulnerability_type
Other                                              376
Privilege Escalation Vulnerability                  52
Arbitrary Code Execution Vulnerability              27
Memory Corruption Vulnerability                     21
Remote Code Execution Vulnerability                 21
Improper Access Control Vulnerability               20
Command Injection Vulnerability                     16
Use-After-Free Vulnerability                        15
Deserialization of Untrusted Data Vulnerability     13
Type Confusion Vulnerability                        12
Path Traversal Vulnerability                        12
Buffer Overflow Vulnerability                       12
Denial of Service Vulnerability                     11
Information Disclosure Vulnerability                11
Out-of-Bounds Write Vulnerability                    9
SQL Injection Vulnerability                          8
Code Injection Vulnerability                         6
Unspecified Vulnerability                     

In [142]:
# # Privilege Escalation Vulnerability  
# search_terms = ["Privilege", "Escalation"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [143]:
# # Memory Corruption Vulnerability
# search_terms = ["Memory", "Corruption"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [ ]:
# # Arbitrary File Upload Vulnerability
# search_terms = ["File", "Upload"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [144]:
# # Remote/Arbitrary Code Execution Vulnerability 
# search_terms = ["Execution", "Remote", "Arbitrary"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [145]:
# # Improper Access Control Vulnerability
# search_terms = ["Access", "Improper", "Control"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [146]:
# # Command Injection Vulnerability
# search_terms = ["Injection", "Command", "SQL"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [147]:
# # Use-After-Free Vulnerability 
# search_terms = ["Use-After", "Use-After-Free"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [148]:
# # Deserialization of Untrusted Data Vulnerability
# search_terms = ["Deserialization", "Untrusted"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [149]:
# # Type Confusion Vulnerability  
# search_terms = ["Type Confusion"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [150]:
# # Path Traversal Vulnerability
# search_terms = ["Traversal", "path"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [151]:
# # Buffer Overflow Vulnerability
# search_terms = ["Buffer", "Overflow", "Heap"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [153]:
# # Denial of Service Vulnerability 
# search_terms = ["Denial", "Service"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [156]:
# # Information Disclosure Vulnerability
# search_terms = ["Information", "Disclosure"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [158]:
# # Out-of-Bounds Write Vulnerability
# search_terms = ["Out-of-Bounds", "Write", "Bounds"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [162]:
# # Sandbox Bypass Vulnerability 
# search_terms = ["Sandbox", "Bypass"]
# pattern = '|'.join(search_terms)

# len(train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)])

In [165]:
# Arbitrary File Upload Vulnerability
search_terms = ["Unspecified"]
pattern = '|'.join(search_terms)

train.vulnerability_name[train['vulnerability_name'].str.contains(pattern, case=False, na=False)]

77      Samsung Mobile Devices Unspecified Vulnerability
360                Apple macOS Unspecified Vulnerability
431     Oracle WebLogic Server Unspecified Vulnerability
456                Apple macOS Unspecified Vulnerability
485             Oracle Java SE Unspecified Vulnerability
494    Adobe Acrobat and Reader Unspecified Vulnerabi...
518    Oracle Fusion Middleware Unspecified Vulnerabi...
534         Adobe Flash Player Unspecified Vulnerability
566    Apple Multiple Products Kernel Unspecified Vul...
570     Oracle WebLogic Server Unspecified Vulnerability
603    Oracle Fusion Middleware Unspecified Vulnerabi...
639     Oracle WebLogic Server Unspecified Vulnerability
Name: vulnerability_name, dtype: object